# 1.1 Amazon SageMaker Training 실습

## 학습 작업의 실행 노트북 개요

이 노트북은 SageMaker에서 Training Job을 통해서 모델 학습합니다.상세한 사항은 개발자 가이드를 참조 하세요. -->  [모델 학습](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script)

- 일반적으로 SageMaker Training은 아래와 같이 진행이 됩니다.
    - [작업 실행 시 필요 라이브러리 import](#작업-실행-시-필요-라이브러리-import)
    - [SageMaker 세션과 Role, 사용 버킷 정의](#SageMaker-세션과-Role,-사용-버킷-정의)
    - [하이퍼파라미터 정의](#하이퍼파라미터-정의)
    - [학습 실행 작업 정의](#학습-실행-작업-정의)
        - 학습 코드 명
        - 학습 코드 폴더 명
        - 학습 코드가 사용한 Framework 종류, 버전 등
        - 학습 인스턴스 타입과 개수
        - SageMaker 세션
        - 학습 작업 하이퍼파라미터 정의
        - 학습 작업 산출물 관련 S3 버킷 설정 등
    - [학습 데이터셋 지정](#학습-데이터셋-지정)
        - 학습에 사용하는 데이터셋의 S3 URI 지정
    - [학습 실행](#학습-실행)
    - [데이터 세트 설명](#데이터-세트-설명)

### 작업 실행 시 필요 라이브러리 import

In [1]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

sudo curl -L "https://github.com/docker/compose/releases/download/v2.7.0/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose

Add data-root and default-shm-size=10G


Redirecting to /bin/systemctl restart docker.service


Docker Restart


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 24.5M  100 24.5M    0     0  61.7M      0 --:--:-- --:--:-- --:--:-- 61.7M


In [2]:
import boto3
import sagemaker

### SageMaker 세션과 Role, 사용 버킷 정의

In [3]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

In [4]:
bucket = sagemaker_session.default_bucket()
code_location = f's3://{bucket}/xgboost/code'
output_path = f's3://{bucket}/xgboost/output'

### 하이퍼파라미터 정의

In [5]:
hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

### 학습 실행 작업 정의

In [6]:
instance_count = 1
instance_type = "ml.m5.large"
instance_type = "local"
max_run = 1*60*60

use_spot_instances = False
if use_spot_instances:
    max_wait = 1*60*60
else:
    max_wait = None

In [7]:
if instance_type in ['local', 'local_gpu']:
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()

In [8]:
from sagemaker.xgboost.estimator import XGBoost

estimator = XGBoost(
    entry_point="xgboost_starter_script.py",
    source_dir='src',
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.3-1",
    max_run=max_run,
    use_spot_instances=use_spot_instances,  # spot instance 활용
    max_wait=max_wait,
)

### 학습 데이터셋 지정

In [9]:
data_path=f's3://{bucket}/xgboost/dataset'
!aws s3 sync ../data/dataset/ $data_path

In [10]:
if instance_type in ['local', 'local_gpu']:
    from pathlib import Path
    file_path = f'file://{Path.cwd()}'
    inputs = file_path.split('lab_1_training')[0] + 'data/dataset/'
    
else:
    inputs = data_path
inputs

'file:///home/ec2-user/SageMaker/h-mbp/simple-sagemaker-base/data/dataset/'

### 학습 실행

In [11]:
estimator.fit(inputs = {'inputdata': inputs},
                  wait=False)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-07-20-04-38-54-227
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-b8p6z:
    command: train
    container_name: 9f1ntlkyev-algo-1-b8p6z
    environment:
    - '[Masked]'
    - '[Masked]'
    image: 246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.3-1
    networks:
      sagemaker-local:
        aliases:
        - algo-1-b8p6z
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmpzi6g5ads/algo-1-b8p6z/input:/opt/ml/input
    - /tmp/tmpzi6g5ads/algo-1-b8p6z/ou

Login Succeeded


INFO:sagemaker.local.image:image pulled: 246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.3-1


Container 9f1ntlkyev-algo-1-b8p6z  Creating
Container 9f1ntlkyev-algo-1-b8p6z  Created
Attaching to 9f1ntlkyev-algo-1-b8p6z
9f1ntlkyev-algo-1-b8p6z  | [2023-07-20 04:39:37.571 f468403c8ce3:1 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
9f1ntlkyev-algo-1-b8p6z  | [2023-07-20 04:39:37.590 f468403c8ce3:1 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
9f1ntlkyev-algo-1-b8p6z  | [2023-07-20:04:39:37:INFO] Imported framework sagemaker_xgboost_container.training
9f1ntlkyev-algo-1-b8p6z  | [2023-07-20:04:39:37:INFO] No GPUs detected (normal if no gpus installed)
9f1ntlkyev-algo-1-b8p6z  | [2023-07-20:04:39:37:INFO] Invoking user training script.
9f1ntlkyev-algo-1-b8p6z  | [2023-07-20:04:39:37:INFO] Module xgboost_starter_script does not provide a setup.py. 
9f1ntlkyev-algo-1-b8p6z  | Generating setup.py
9f1ntlkyev-algo-1-b8p6z  | [2023-07-20:04:39:37:INFO] Generating setup.cfg
9f1ntlkyev-algo-1-b8p6z  | [202

INFO:root:creating /tmp/tmpzi6g5ads/artifacts/output/data
INFO:root:copying /tmp/tmpzi6g5ads/algo-1-b8p6z/output/data/metrics.json -> /tmp/tmpzi6g5ads/artifacts/output/data
INFO:root:copying /tmp/tmpzi6g5ads/model/xgboost-model -> /tmp/tmpzi6g5ads/artifacts/model


9f1ntlkyev-algo-1-b8p6z exited with code 0
Aborting on container exit...
Container 9f1ntlkyev-algo-1-b8p6z  Stopping
Container 9f1ntlkyev-algo-1-b8p6z  Stopped
time="2023-07-20T04:39:42Z" level=error msg=0
===== Job Complete =====


In [12]:
estimator.logs()

2023-07-20 03:58:54 Starting - Starting the training job...
2023-07-20 03:59:08 Starting - Preparing the instances for training......
2023-07-20 04:00:00 Downloading - Downloading input data...
2023-07-20 04:00:35 Training - Downloading the training image...
2023-07-20 04:01:16 Training - Training image download completed. Training in progress..[2023-07-20 04:01:21.630 ip-10-0-166-216.us-west-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-07-20 04:01:21.654 ip-10-0-166-216.us-west-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-07-20:04:01:21:INFO] Imported framework sagemaker_xgboost_container.training
[2023-07-20:04:01:21:INFO] No GPUs detected (normal if no gpus installed)
[2023-07-20:04:01:21:INFO] Invoking user training script.
[2023-07-20:04:01:21:INFO] Module xgboost_starter_script does not provide a setup.py. 
Generating setup.py
[2023-07-20:04:01:21:INFO] Generating setup.cfg
[2023-07-20:04:01:21:INFO

### 데이터 세트 설명
- 데이터 세트는 블로그 [Architect and build the full machine learning lifecycle with AWS: An end-to-end Amazon SageMaker demo](https://aws.amazon.com/blogs/machine-learning/architect-and-build-the-full-machine-learning-lifecycle-with-amazon-sagemaker/) 에서 사용한 데이터를 사용합니다. 블로그에서는 데이터 세트에 대해서 이렇게 설명 합니다.
- "자동차 보험 청구 사기를 탐지를 위해서 블로그 저자가 데이터를 합성해서 만든 고객과 클레임의 데이터 세트를 사용합니다."

In [13]:
import pandas as pd

In [14]:
train_prep_df = pd.read_csv('../data/dataset/train.csv')
train_prep_df.groupby('fraud').sample(n=5)

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
578,0,7254.641372,14354.641372,18,4,0,1,750,3000,90803,...,0,0,1,0,0,0,1,0,1,0
1432,0,11310.813645,12810.813645,63,199,0,1,750,2650,94123,...,0,1,0,0,0,0,0,1,0,1
861,0,1000.000000,34200.000000,55,180,0,1,750,3000,83221,...,0,0,1,0,0,0,1,0,1,0
2097,0,34929.516483,68029.516483,36,35,0,2,750,3000,97224,...,0,0,0,1,0,0,0,1,0,1
1735,0,15454.960951,17854.960951,60,137,0,1,750,2600,91915,...,0,0,1,0,0,0,1,0,1,0
3587,1,6250.360195,10850.360195,36,73,0,1,750,2950,85716,...,0,0,0,1,0,0,0,1,0,1
3677,1,34743.050328,53843.050328,26,26,0,2,750,3000,92128,...,0,0,0,1,0,0,0,1,1,0
1688,1,5481.401810,14381.401810,32,66,0,1,750,3000,89801,...,0,0,1,0,0,0,1,0,1,0
3043,1,36334.783453,45534.783453,60,127,0,1,750,3000,92691,...,0,0,0,1,0,1,0,0,1,0
2311,1,17523.717885,25123.717885,61,76,0,1,750,2450,92104,...,1,1,0,0,0,0,0,1,1,0


In [15]:
train_prep_df.groupby('fraud').size()

fraud
0    3869
1     131
dtype: int64

#### 데이터 컬럼 설명
- fraud: 보험 청구의 사기 여부 입니다. 1 이면 사기, 0 이면 정상 청구 입니다.
- vehicle_claim: 자동차에 대한 보험 청구액. 값으로서, $1000, $17,638 등이 있습니다.
- total_claim_amount: 전체 보험 청구액 입니다. $21,400, $10,000 등이 있습니다.    
- customer_age: 고객의 나이를 의미 합니다.
- months_as_customer: 고객으로서의 가입 기간을 의미합니다. 단위는 월로서 11, 30, 31 등의 값이 존재 합니다.
- num_claims_past_year: 작년의 보험 청구 수를 의미 합니다. 0, 1, 2, 3, 4, 5, 6 의 값이 존재 합니다.
- num_insurers_past_5_years: 과거 5년 동안의 보험 가입 회사 수를 의미 합니다. 1, 2, 3, 4, 5 의 값이 존재 합니다.
- policy_deductable: 보험의 최소 자기 부담금 입니다. $750, $800 등이 있습니다.    
- policy_annual_premium: 보험의 특약 가입에 대한 금액 입니다. $2000, $3000 등이 있습니다.
- customer_zip: 고객의 집 주소 우편 번호를 의미합니다.
- auto_year: 자동차의 년식을 의미 합니다. 2020, 2019 등이 있습니다.
- num_vehicles_involved: 몇 대의 자동차가 사고에 연관 되었는지 입니다. 1, 2, 3, 4, 5, 6 의 값이 있습니다.
- num_injuries: 몇 명이 상해를 입었는지를 기술합니다. 0, 1, 2, 3, 4, 의 값이 있습니다.
- num_witnesses: 몇 명의 목격자가 있었는지를 기술합니다. 0, 1, 2, 3, 4, 5 의 값이 있습니다.
- injury_claim: 상해에 대한 보험 청구액. \$5,500, \$70,700, \$100,700 등이 있습니다.    
- incident_month: 사고가 발생한 월을 의미합니다. 1~12 값이 존재 합니다.
- incident_day: 사고가 발생한 일자를 의미합니다. 1~31 값이 존재 합니다.
- incident_dow: 사고가 발생한 요일을 의미합니다. 0~6 값이 존재 합니다.
- incident_hour: 사고가 발생한 시간을 의미합니다. 0~23 값이 존재 합니다.
- policy_state: 보험 계약을 한 미국 주(State)를 의미 합니다. CA, WA, AZ, OR, NV, ID 가 존재 합니다.    
- policy_liability: 보험 청구의 한도를 의미 합니다. 예를 들어서 25/50 은  사람 당 상해 한도 $25,000, 사고 당 상해 한도가 $50,000 을  의미합니다. 25/50, 15/30, 30/60, 100/200 의 값이 존재 합니다. 
- customer_gender: 고객의 성별을 의미 합니다. Male, Female, Unkown, Other가 존재 합니다.
- customer_education: 고객의 최종 학력을 의미합니다. Bachelor, High School, Advanced Degree, Associate, Below High School 이 존재 합니다.
- driver_relationship: 보험 계약자와 운전자와의 관계 입니다. Self, Spouse, Child, Other 값이 존재 합니다.
- incident_type: 사고의 종류를 기술합니다. Collision, Break-in, Theft 값이 존재 합니다.
- collision_type: 충돌 타입을 기술합니다. Front, Rear, Side, missing 값이 존재 합니다.
- incident_severity: 사고의 손실 정도 입니다. Minor, Major, Totaled 값이 존재 합니다.
- authorities_contacted: 어떤 관련 기관에 연락을 했는지 입니다. Police, Ambuylance, Fire, None 값이 존재 합니다.
- police_report_available: 경찰 보고서가 존재하는지를 기술합니다. Yes, No 의 값이 있습니다.